In [ ]:
import os
import time
import shutil
import warnings
import time

# Suppress future warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Create constraints file
!echo "numpy<2.0" > /kaggle/working/constraints.txt

# Set environment variable for the entire notebook
import os
os.environ['PIP_CONSTRAINT'] = '/kaggle/working/constraints.txt'

# --------------------------
# 0. 清理旧进程
# --------------------------
print("清理旧进程...")
!pkill -f 'python main.py' || true
!pkill -f 'pinggy' || true

# --------------------------
# 0.5 检测和配置多GPU
# --------------------------
print("\n" + "="*60)
print("检测 GPU 配置...")
print("="*60)

try:
    import torch
    num_gpus = torch.cuda.device_count()
    print(f"✓ 检测到 {num_gpus} 个 GPU")
    
    for i in range(num_gpus):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        print(f"    - 显存: {props.total_memory / 1024**3:.2f} GB")
    
    # 设置环境变量以启用多GPU
    if num_gpus > 1:
        print(f"\n✓ 配置多GPU模式 ({num_gpus} GPUs)")
        os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, range(num_gpus)))
    else:
        print("\n⚠ 仅检测到1个GPU")
        
except Exception as e:
    print(f"✗ GPU检测失败: {e}")

print("="*60)

# --------------------------
# 1. 系统依赖
# --------------------------
print("安装系统库...")
!apt-get update -qq
!apt-get install -y -qq libgl1 libglib2.0-0 ffmpeg

# --------------------------
# 2. 克隆 ComfyUI
# --------------------------
print("克隆 ComfyUI 仓库...")
if not os.path.exists("ComfyUI"):
    !git clone https://github.com/comfyanonymous/ComfyUI.git

%cd ComfyUI
print(f"当前工作目录: {os.getcwd()}")
print("更新 ComfyUI...")
!git pull

# --------------------------
# 3. 安装 Python 库 (优化版本组合)
# --------------------------
print("安装额外依赖...")
!pip install -q mediapipe
!pip uninstall onnxruntime -y
!pip install onnxruntime-gpu

import onnxruntime as ort
print(ort.get_available_providers())
# Should show: ['CUDAExecutionProvider', 'CPUExecutionProvider']

# SAM-2 needs to be installed from git
!pip install -q git+https://github.com/facebookresearch/segment-anything-2.git

print("卸载不兼容库...")
!pip uninstall -y torch torchvision torchaudio xformers

print("安装最新 PyTorch nightly (WanVideoWrapper 要求 2.7.0.dev20250226+)...")
!pip install --pre --upgrade --extra-index-url https://download.pytorch.org/whl/nightly/cu121 torch torchvision torchaudio

print("安装兼容的 xformers...")
!pip install -q xformers

print("安装 ComfyUI 其余依赖...")

# Downgrade numpy to avoid incompatibility with custom nodes

!pip install -q -r requirements.txt --no-deps
!pip install -q torchsde einops transformers safetensors aiohttp accelerate pyyaml opencv-python pillow scipy imageio[ffmpeg] moviepy huggingface_hub gguf
!pip install -q pydantic-settings python-dotenv

# --------------------------
# 4. 验证安装
# --------------------------
print("\n" + "="*50)
print("验证 PyTorch 和 xformers...")
print("="*50)
try:
    import torch
    import torchvision
    import xformers
    import xformers.ops
    
    print(f"✓ PyTorch: {torch.__version__}")
    print(f"✓ TorchVision: {torchvision.__version__}")
    print(f"✓ xformers: {xformers.__version__}")
    print(f"✓ CUDA 可用: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"✓ GPU 数量: {num_gpus}")
        for i in range(num_gpus):
            print(f"  - GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"✓ CUDA 版本: {torch.version.cuda}")
    
    # 验证 RMSNorm 可用性
    if hasattr(torch.nn, 'RMSNorm'):
        print("✓ RMSNorm 可用 (WanVideoWrapper 将正常工作)")
    else:
        print("✗ RMSNorm 不可用")
        
    print("="*50)
except Exception as e:
    print(f"✗ 验证失败: {e}")
    print("="*50)

# --------------------------
# 5. 使用 Kaggle Input 的模型 (通过符号链接)
# --------------------------
kaggle_input_dir = "/kaggle/input"

print("\n使用本地 Kaggle Input 模型目录:", kaggle_input_dir)

# 创建模型目录
model_dirs = {
    "diffusion_models": "./models/diffusion_models",
    "text_encoders": "./models/text_encoders",
    "vae": "./models/vae",
    "loras": "./models/loras",
    "clip_vision":"./models/clip_vision"
}

for dir_path in model_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# 使用符号链接代替复制文件
print("\n创建模型文件符号链接 (节省存储空间)...")
print("="*60)

link_map = {
    "wan-animate-gguf-text-encoder": "./models/text_encoders",
    "wan-animate-vae": "./models/vae", 
    "wan-animate-q3-k-m-gguf": "./models/unet",
    "wan-animate-loras": "./models/loras",
    "wan-animate-clip-vision":"./models/clip_vision"
}

for src_name, dst_path in link_map.items():
    src_path = f"{kaggle_input_dir}/{src_name}"
    if os.path.exists(src_path):
        print(f"\n处理: {src_name}")
        try:
            # 列出源目录中的文件
            files = os.listdir(src_path)
            if not files:
                print(f"  ⚠ 空目录: {src_name}")
                continue
            
            # 为每个文件创建符号链接
            for file in files:
                src_file = os.path.join(src_path, file)
                dst_file = os.path.join(dst_path, file)
                
                # 如果目标已存在，先删除
                if os.path.exists(dst_file) or os.path.islink(dst_file):
                    os.remove(dst_file)
                    print(f"  - 删除旧链接: {file}")
                
                # 创建符号链接
                os.symlink(src_file, dst_file)
                print(f"  ✓ 链接: {file}")
                
        except Exception as e:
            print(f"  ✗ 错误: {e}")
    else:
        print(f"  ⚠ 未找到: {src_name}")

print("\n" + "="*60)
print("模型文件链接列表:")
print("-" * 50)
for name, path in model_dirs.items():
    print(f"\n{name}:")
    !ls -lh {path}/ 2>/dev/null || echo "  (空)"

# 验证符号链接
print("\n验证符号链接...")
for name, path in model_dirs.items():
    if os.path.exists(path):
        files = os.listdir(path)
        for file in files:
            full_path = os.path.join(path, file)
            if os.path.islink(full_path):
                target = os.readlink(full_path)
                print(f"  ✓ {file} -> {target}")

# --------------------------
# 6. 安装所有必需插件
# --------------------------
print("\n" + "="*60)
print("安装 ComfyUI 插件...")
print("="*60)

os.makedirs("custom_nodes", exist_ok=True)
%cd custom_nodes

# Complete list of plugins including Kijai workflow requirements
all_plugins = {
    # Core plugins
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "ComfyUI-GGUF": "https://github.com/city96/ComfyUI-GGUF.git",
    "ComfyUI-WanVideoWrapper": "https://github.com/kijai/ComfyUI-WanVideoWrapper.git",
    
    # Kijai workflow requirements
    "ComfyUI-KJNodes": "https://github.com/kijai/ComfyUI-KJNodes.git",
    "comfyui_controlnet_aux": "https://github.com/Fannovel16/comfyui_controlnet_aux.git",
    "ComfyUI-VideoHelperSuite": "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git",
    "ComfyUI-segment-anything-2": "https://github.com/kijai/ComfyUI-segment-anything-2.git",
    "ComfyUI-Advanced-ControlNet": "https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet.git",
    "comfyui-tooling-nodes": "https://github.com/Acly/comfyui-tooling-nodes.git",
    "ComfyUI_essentials": "https://github.com/cubiq/ComfyUI_essentials.git",
    
    # Additional required nodes
    "cg-use-everywhere": "https://github.com/chrisgoringe/cg-use-everywhere.git",
    "was-node-suite-comfyui": "https://github.com/WASasquatch/was-node-suite-comfyui.git",
    "ComfyUI-Custom-Scripts": "https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git",
    "ComfyUI-Frame-Interpolation": "https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git",
    "rgthree-comfy": "https://github.com/rgthree/rgthree-comfy.git",
    "ComfyMath": "https://github.com/evanspearman/ComfyMath.git",
    "ComfyUI-Easy-Use": "https://github.com/yolain/ComfyUI-Easy-Use.git",
    "comfyui-mixlab-nodes": "https://github.com/MixLabPro/comfyui-mixlab-nodes.git"
}

for plugin_name, plugin_url in all_plugins.items():
    print(f"\n{'='*60}")
    print(f"处理: {plugin_name}")
    print(f"{'='*60}")
    
    if not os.path.exists(plugin_name):
        print(f"  → 克隆 {plugin_name}...")
        !git clone {plugin_url}
        
        # Install requirements
        req_file = f"{plugin_name}/requirements.txt"
        if os.path.exists(req_file):
            print(f"  → 安装依赖...")
            !pip install -q -r {req_file}
            
        print(f"  ✓ {plugin_name} 完成")
    else:
        print(f"  → {plugin_name} 已存在，更新中...")
        %cd {plugin_name}
        !git pull -q
        
        # Update requirements
        if os.path.exists("requirements.txt"):
            !pip install -q -r requirements.txt
        %cd ..
        print(f"  ✓ {plugin_name} 更新完成")

%cd ..
print("\n" + "="*60)
print(f"✓ 所有插件安装完成！")
print(f"当前工作目录: {os.getcwd()}")
print("="*60)

# --------------------------
# 6.5 配置 ComfyUI 多GPU支持
# --------------------------
print("\n" + "="*60)
print("配置 ComfyUI 多GPU支持...")
print("="*60)

# Remove or fix the extra_model_paths.yaml file to prevent startup errors
config_file = "extra_model_paths.yaml"
if os.path.exists(config_file):
    os.remove(config_file)
    print(f"✓ 已删除旧配置文件: {config_file}")

# ComfyUI will use default paths, which works better with multi-GPU
print("✓ 使用默认配置 (自动多GPU支持)")
print("="*60)

# --------------------------
# 7. 下载 Localhost 并启动 ComfyUI Web UI
# --------------------------
print("\n" + "="*60)
print("下载 Localhost 并启动 ComfyUI Web UI...")
print("="*60)

# Suppress warnings in environment
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore::FutureWarning'

# Change to ComfyUI directory
%cd /kaggle/working/ComfyUI

print("\n🚀 启动 ComfyUI 服务器 (多GPU模式)...")

# Check GPU count one more time
try:
    import torch
    num_gpus = torch.cuda.device_count()
    if num_gpus > 1:
        print(f"✓ 启用 {num_gpus} GPU 模式")
        print(f"  CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES', 'all')}")
    print()
except:
    pass

!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --output-directory kaggle/working/output

清理旧进程...

检测 GPU 配置...
✓ 检测到 2 个 GPU
  GPU 0: Tesla T4
    - 显存: 14.74 GB
  GPU 1: Tesla T4
    - 显存: 14.74 GB

✓ 配置多GPU模式 (2 GPUs)
安装系统库...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
克隆 ComfyUI 仓库...
Cloning into 'ComfyUI'...
remote: Enumerating objects: 26834, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 26834 (delta 65), reused 45 (delta 37), pack-reused 26744 (from 3)
Receiving objects: 100% (26834/26834), 74.36 MiB | 28.22 MiB/s, done.
Resolving deltas: 100% (18094/18094), done.
/kaggle/working/ComfyUI/ComfyUI
当前工作目录: /kaggle/working/ComfyUI/ComfyUI
更新 ComfyUI...
Already up to date.
安装额外依赖...
Found existing installation: onnxruntime 1.23.2
Uninstalling onnxruntime-1.23.2:
  Successfully uninstalled onnxruntime-1.23.2
['TensorrtExecutionProvider', 'CUDAExecutionProvider

/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:33777 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0


FETCH ComfyRegistry Data: 10/107
FETCH ComfyRegistry Data: 15/107
FETCH ComfyRegistry Data: 20/107
FETCH ComfyRegistry Data: 25/107
FETCH ComfyRegistry Data: 30/107
FETCH ComfyRegistry Data: 35/107
FETCH ComfyRegistry Data: 40/107
FETCH ComfyRegistry Data: 45/107
FETCH ComfyRegistry Data: 50/107
FETCH ComfyRegistry Data: 55/107
FETCH ComfyRegistry Data: 60/107
FETCH ComfyRegistry Data: 65/107
FETCH ComfyRegistry Data: 70/107
FETCH ComfyRegistry Data: 75/107
FETCH ComfyRegistry Data: 80/107
FETCH ComfyRegistry Data: 85/107
FETCH ComfyRegistry Data: 90/107
FETCH ComfyRegistry Data: 95/107
FETCH ComfyRegistry Data: 100/107
FETCH ComfyRegistry Data: 105/107
FETCH ComfyRegistry Data [DONE]
[ComfyUI-Manager] default cache updated: https://api.comfy.org/nodes
FETCH DATA from: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json [DONE]
[ComfyUI-Manager] All startup tasks have been completed.
